In [1]:
import os
import time
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nfl-playoffs-19702021/df_kaggle_test.csv
/kaggle/input/nfl-playoffs-19702021/playoffs.csv
/kaggle/input/nfl-playoffs-19702021/all_games.csv
/kaggle/input/nfl-playoffs-19702021/df_kaggle_test_labels.csv
/kaggle/input/nfl-playoffs-19702021/df_kaggle_train.csv
/kaggle/input/nfl-playoffs-19702021/games_calculated_averages.csv
/kaggle/input/nfl-playoffs-19702021/1970-2021.csv
/kaggle/input/nfl-playoffs-19702021/regular_season.csv
/kaggle/input/nfl-playoffs-19702021/1970-2021_team_schedule.csv
/kaggle/input/nfl-playoffs-19702021/df_only_averages.csv
/kaggle/input/nfl-playoffs-19702021/grouped_averages.csv


In [4]:
class Timer:
    
    def __init__(self):
        self.start_time = time.time()
        
    def end(self):
        self.end_time = time.time()
        elapsed_time = round(self.end_time - self.start_time, 3)
        print(f'Execution took {elapsed_time}s')

---

# Tentativa com variaveis temporais

In [ ]:
# df_1970_2021csv = pd.read_csv('/kaggle/input/nfl-playoffs-19702021/1970-2021.csv', sep=';')
# df_all_gamescsv = pd.read_csv('/kaggle/input/nfl-playoffs-19702021/all_games.csv', sep=';')

In [ ]:
# df = df_all_gamescsv.copy()

# # only home games
# df.drop(df[df['gs_game_location'] == 'away'].index, inplace=True)
# df.drop('Unnamed: 0', axis=1, inplace=True)

# # df = df.loc[df['gs_year'] == 2019]
# df.sort_values(['gs_week_number', 'gs_id'], inplace=True)

In [ ]:
# # game stat columns
# gs_cols = df.columns[df.columns.str.startswith('gs_')].values.tolist()

# gs_num_cols = df.select_dtypes(np.number).columns \
#     [df.select_dtypes(np.number).columns.str.startswith('gs_')] \
#     .values.tolist()
# gs_num_cols.remove('gs_id')
# gs_num_cols.remove('gs_year')
# gs_num_cols.remove('gs_week_number')

average from all games that already happened

values from last season

group by season and team, order by gs_id

calculate average stats for all games that happened before

## Calculo das medias

In [ ]:
# def get_average_so_far(x):
#     averages = df.query('gs_year == @x.name[0] and gs_team == @x.name[1] and gs_week_number < @x.name[2]')[gs_num_cols].mean()
#     return averages

In [ ]:
# timer = Timer()

# grouped_averages = df.groupby(['gs_year', 'gs_team', 'gs_week_number'])[gs_num_cols].apply(get_average_so_far)
# new_average_column_names = {old_col: 'avg_' + old_col for old_col in grouped_averages.columns}
# grouped_averages.rename(new_average_column_names, axis=1, inplace=True)

# timer.end()

In [ ]:
# grouped_averages.reset_index().to_csv('grouped_averages.csv')

In [ ]:
# df_with_averages = df.join(grouped_averages, on=['gs_year', 'gs_team', 'gs_week_number'])
# df_with_averages.to_csv('games_calculated_averages.csv')

In [ ]:
# df_with_averages.shape

In [5]:
df_with_averages = pd.read_csv('/kaggle/input/nfl-playoffs-19702021/games_calculated_averages.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [36]:
new_df = df_with_averages.copy()

if 'Unnamed: 0' in new_df:
    new_df.drop('Unnamed: 0', axis=1, inplace=True)

## Nomes dos times

Ignorando o nome da cidade

In [37]:
def get_team_last_name(x):
    return x.split()[-1]

In [38]:
new_df['gs_team'] = new_df['gs_team'].apply(get_team_last_name)
new_df['gs_opp'] = new_df['gs_opp'].apply(get_team_last_name)

## Dados do ultimo ano

In [39]:
# gs_avg_cols = list(new_average_column_names.values())
gs_avg_cols = new_df.columns[new_df.columns.str.startswith('avg_gs_')].tolist()

In [40]:
home_team_stat_cols = new_df.columns[new_df.columns.str.startswith('hts_')].tolist()
away_team_stat_cols = new_df.columns[new_df.columns.str.startswith('ats_')].tolist()

In [41]:
last_year_cols = ['last_year_' + col for col in gs_avg_cols]
gs_last_year_avg_cols = last_year_cols

In [42]:
grouped_by_year_team = new_df.sort_values('gs_year') \
    .groupby(['gs_year', 'gs_team'])[gs_avg_cols] \
    .mean()

In [55]:
# parece simples, mas isso aqui me deu uma canseira
_df = new_df.sort_values('gs_year').copy()

_df.loc[:, last_year_cols] = np.nan
_df.set_index(['gs_year', 'gs_team'], inplace=True)

for season in _df.index.get_level_values(0).unique().tolist()[1:]:
    for team in _df.index.get_level_values(1).unique().tolist():
        try:
            last_year_averages = grouped_by_year_team.loc[(season - 1, team), :]
        except KeyError:
            continue
        _df.loc[(season, team), last_year_cols] = last_year_averages.values

/opt/conda/lib/python3.7/site-packages/IPython/core/async_helpers.py:78: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2975: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures, cell_id


In [56]:
_df[['gs_game_outcome'] + gs_avg_cols + gs_last_year_avg_cols] \
    .loc[(2016, 'Bills')]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  


gs_game_outcome  avg_gs_pts_off  avg_gs_pts_def  avg_gs_first_down_off  avg_gs_yards_off  avg_gs_pass_yds_off  avg_gs_rush_yds_off  avg_gs_to_off  avg_gs_first_down_def  avg_gs_yards_def  avg_gs_pass_yds_def  avg_gs_rush_yds_def  avg_gs_to_def  avg_gs_exp_pts_off  avg_gs_exp_pts_def  avg_gs_exp_pts_st  last_year_avg_gs_pts_off  last_year_avg_gs_pts_def  last_year_avg_gs_first_down_off  last_year_avg_gs_yards_off  last_year_avg_gs_pass_yds_off  last_year_avg_gs_rush_yds_off  last_year_avg_gs_to_off  last_year_avg_gs_first_down_def  last_year_avg_gs_yards_def  last_year_avg_gs_pass_yds_def  last_year_avg_gs_rush_yds_def  last_year_avg_gs_to_def  last_year_avg_gs_exp_pts_off  last_year_avg_gs_exp_pts_def  last_year_avg_gs_exp_pts_st
gs_year gs_team                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
2016    Bills                 W       11.500000        6.500000              17.500000        269.000000           169.500000            99.500000            NaN              15.500000        292.500000           206.000000            86.500000       1.000000           -0.670000            7.670000          -3.625000                 27.926531                 18.548299                        18.843878                  344.997959                     210.506463                     134.491497                      2.0                         18.57551                  339.470748                     231.452721                     108.018027                 2.107143                      3.985422                      4.821629                     0.359378
        Bills                 L             NaN             NaN                    NaN               NaN                  NaN                  NaN            NaN                    NaN               NaN                  NaN                  NaN            NaN                 NaN                 NaN                NaN                 27.926531                 18.548299                        18.843878                  344.997959                     210.506463                     134.491497                      2.0                         18.57551                  339.470748                     231.452721                     108.018027                 2.107143                      3.985422                      4.821629                     0.359378
        Bills                 L       20.428571       20.142857              20.000000        322.714286           177.571429           145.142857       1.333333              20.857143        337.285714           223.571429           113.714286       1.750000            3.555714           -2.695714          -1.190000                 27.926531                 18.548299                        18.843878                  344.997959                     210.506463                     134.491497                      2.0                         18.57551                  339.470748                     231.452721                     108.018027                 2.107143                      3.985422                      4.821629                     0.359378
        Bills                 W       20.600000       18.200000              19.200000        307.000000           182.800000           124.200000       1.000000              19.800000        332.400000           219.600000           11

## Gerando features para o time visitante

In [59]:
_old_df = _df.copy()

In [57]:
gs_info_cols = ['gs_opp', 'gs_id', 'gs_week_number', 'gs_is_playoff', 'gs_game_outcome']
df_averages = _df[gs_info_cols + gs_avg_cols + gs_last_year_avg_cols] \
    .reset_index() \
    .rename({'gs_game_outcome': 'winorlose'}, axis=1)

df_averages.to_csv('grouped_averages.csv')

In [69]:
all_gs_cols = gs_avg_cols + gs_last_year_avg_cols

new_hts_cols_remap = {old_col: 'hts_' + old_col for old_col in all_gs_cols}

new_hts_cols = ['hts_' + old_col for old_col in all_gs_cols]
new_ats_cols = ['ats_' + old_col for old_col in all_gs_cols]

In [94]:
def get_opp_stats(x):
    year = x.name[0]
    team = x['gs_opp']
    week = x.name[2]

    return _df.loc[(year, team, week), new_hts_cols]

In [146]:
_df = df_averages.copy()

# _df.set_index(['gs_year', 'gs_team', 'gs_week_number'], inplace=True)
_df.rename(new_hts_cols_remap, axis=1, inplace=True)
_df['new_opp'] = _df['gs_opp']

# _df.loc[:, new_ats_cols] = _df.loc

# for opp in _df['gs_opp'].unique().tolist():

In [159]:
_df_opp = _df.copy()[['gs_id', 'gs_team'] + new_hts_cols] # copy home team
_df_opp['new_opp'] = _df_opp['gs_team']
_df_opp.drop('gs_team', axis=1, inplace=True)

# _df_opp.index.names = ['gs_year', 'gs_opp', 'gs_week_number'] # set home team to be opp, so we can reuse stats

new_ats_cols_remap = {old_col: old_col.replace('hts_', 'ats_') for old_col in new_hts_cols} # rename hts to ats
_df_opp.rename(new_ats_cols_remap, axis=1, inplace=True)

In [161]:
df_averages_hts_ats = _df.set_index(['gs_id', 'new_opp']) \
    .join(_df_opp.set_index(['gs_id', 'new_opp']), on=['gs_id', 'new_opp'])

In [166]:
df_averages_hts_ats.reset_index() \
    [['gs_team', 'gs_opp', 'gs_year', 'gs_week_number',
      'ats_avg_gs_pts_off', 'ats_avg_gs_pts_def', 'ats_avg_gs_first_down_off']] \
    .query('gs_opp == "49ers" and gs_year == 2021')

,gs_team,gs_opp,gs_year,gs_week_number,ats_avg_gs_pts_off,ats_avg_gs_pts_def,ats_avg_gs_first_down_off
12326,Seahawks,49ers,2021,3.0,NaN,NaN,NaN
12336,Packers,49ers,2021,2.0,NaN,NaN,NaN
12434,Vikings,49ers,2021,11.0,NaN,NaN,NaN
12449,Texans,49ers,2021,16.0,NaN,NaN,NaN
12483,Falcons,49ers,2021,14.0,NaN,NaN,NaN
12502,Colts,49ers,2021,6.0,NaN,NaN,NaN
12554,Rams,49ers,2021,9.0,NaN,NaN,NaN
12594,Cardinals,49ers,2021,8.0,NaN,NaN,NaN


## Observaçoes

In [ ]:
_df[pd.isna(_df['gs_game_location'])]

---